# Run experiments with py_experimenter
This involves specifying different active learning pipelines and evaluating them on different active learning problems. The results are then stored in a database. To specify the exact experiments and parameters to be run, the file config/exp_config.yml is used as well as parameters are filled in the function experimenter.fill_table_from_combination() below. In our case, the results will be filled into ALPBenchmark.db

For further information we refer to the docs https://tornede.github.io/py_experimenter/

In [1]:
import numpy as np
from py_experimenter.exceptions import DatabaseConnectionError
from py_experimenter.experimenter import PyExperimenter, ResultProcessor
import types
from alpbench.benchmark.BenchmarkConnector import DataFileBenchmarkConnector
from alpbench.benchmark.BenchmarkSuite import TabZillaBenchmarkSuite
from alpbench.evaluation.experimenter.DefaultSetup import ensure_default_setup
from alpbench.evaluation.experimenter.LogTableObserver import LogTableObserver, SparseLogTableObserver
from alpbench.pipeline.ActiveLearningPipeline import ActiveLearningPipeline
from alpbench.pipeline.Oracle import Oracle
import sqlite3
import pandas as pd

### get ids of the **tabzilla benchmark suite**

In [2]:
tabzilla = TabZillaBenchmarkSuite()
tabzilla_ids = tabzilla.get_openml_dataset_ids()

### setup experiment runner


In [3]:
# loads parameters from the grid and runs each active learning pipeline on every active learning problem
class ExperimentRunner:

    def __init__(self):
        pass

    def run_experiment(self, parameters: dict, result_processor: ResultProcessor, custom_config: dict):

        dbbc = DataFileBenchmarkConnector()

        connector: DataFileBenchmarkConnector = dbbc

        OPENML_ID = int(parameters["openml_id"])
        SETTING_NAME = parameters["setting_name"]
        TEST_SPLIT_SEED = int(parameters["test_split_seed"])
        TRAIN_SPLIT_SEED = int(parameters["train_split_seed"])
        SEED = int(parameters["seed"])

        setting = connector.load_setting_by_name(SETTING_NAME)
        scenario = connector.load_or_create_scenario(
            openml_id=OPENML_ID,
            test_split_seed=TEST_SPLIT_SEED,
            train_split_seed=TRAIN_SPLIT_SEED,
            seed=SEED,
            setting_id=setting.get_setting_id(),
        )

        X_l, y_l, X_u, y_u, X_test, y_test = scenario.get_data_split()

        QUERY_STRATEGY = connector.load_query_strategy_by_name(parameters["query_strategy_name"])
        LEARNER = connector.load_learner_by_name(parameters["learner_name"])

        OBSERVER = [SparseLogTableObserver(result_processor, X_test, y_test)]

        ALP = ActiveLearningPipeline(
            learner=LEARNER,
            query_strategy=QUERY_STRATEGY,
            observer_list=OBSERVER,
            num_iterations=setting.get_number_of_iterations(),
            num_queries_per_iteration=setting.get_number_of_queries(),
        )

        oracle = Oracle(X_u, y_u)
        ALP.active_fit(X_l, y_l, X_u, oracle)

### run default setup

In [4]:
# , choose learning algorithms, query strategies and parameters **(chosen algorithms and their parameters
# are saved in alpbench/ in .json files**, to not make these experiments run for too long, we restrict
# ourselves to the first **4 dataset ids, 1 setting, 1 seed, 2 learning algorithms and 2 query strategies**


def run(run_setup=False, reset_experiments=False):

    exp_config_file = "config/exp_config.yml"

    experimenter = PyExperimenter(experiment_configuration_file_path=exp_config_file)

    if run_setup:
        benchmark_connector = DataFileBenchmarkConnector()
        ensure_default_setup(dbbc=benchmark_connector)

        benchmark_connector.cleanup()

        setting_combinations = []
        setting_combinations += [{"setting_name": "small"}]

        if reset_experiments:
            experimenter.reset_experiments("running", "failed")

        else:
            experimenter.fill_table_from_combination(
                parameters={
                    "learner_name": ["rf_entropy", "svm_rbf"],  # "xgb", "knn_3", "mlp",
                    # "tabpfn", "tabnet", "catboost"],
                    "query_strategy_name": [  # "core_set", "falcun",
                        "margin",
                        # "least_confident", "entropy",
                        # "power_margin", "bald",
                        # "power_bald",
                        # "max_entropy",
                        # "qbc_variance_ratio", "kmeans",
                        # "cluster_margin", "typ_cluster",
                        # "weighted_cluster",
                        "random",
                    ],
                    "test_split_seed": np.arange(1),
                    "train_split_seed": np.arange(1),
                    "seed": np.arange(1),
                    "openml_id": tabzilla_ids[:1],
                },
                fixed_parameter_combinations=setting_combinations,
            )

    else:
        er = ExperimentRunner()
        experimenter.execute(er.run_experiment, -1)

In [5]:
run(run_setup=True, reset_experiments=False)

2024-06-13 10:35:24,459  | py-experimenter - WARNING  | No values given for keyfield setting_name
2024-06-13 10:35:24,461  | py-experimenter - WARNING  | No values given for keyfield openml_id
2024-06-13 10:35:24,463  | py-experimenter - WARNING  | No values given for keyfield learner_name
2024-06-13 10:35:24,463  | py-experimenter - WARNING  | No values given for keyfield query_strategy_name
2024-06-13 10:35:24,464  | py-experimenter - WARNING  | No values given for keyfield test_split_seed
2024-06-13 10:35:24,465  | py-experimenter - WARNING  | No values given for keyfield train_split_seed
2024-06-13 10:35:24,465  | py-experimenter - WARNING  | No values given for keyfield seed
2024-06-13 10:35:24,466  | py-experimenter - INFO     | Found 7 keyfields
2024-06-13 10:35:24,467  | py-experimenter - WARNING  | No resultfields given
2024-06-13 10:35:24,468  | py-experimenter - INFO     | Found 2 logtables
2024-06-13 10:35:24,469  | py-experimenter - INFO     | Found logtable results__accur

### display (empty) tables

In [6]:
# Specify the path to your .db file
db_path = "ALPBenchmark.db"

# Connect to the database
conn = sqlite3.connect(db_path)

# Create a cursor object to interact with the database
cursor = conn.cursor()

# Get the list of tables in the database
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()

# Display the contents of each table
for table_name in tables:
    table_name = table_name[0]
    print(f"Contents of table {table_name}:")
    query = f"SELECT * FROM {table_name}"
    df = pd.read_sql_query(query, conn)
    display(df)
    print("\n")

# Close the connection
conn.close()

Contents of table results:


,ID,setting_name,openml_id,learner_name,query_strategy_name,test_split_seed,train_split_seed,seed,creation_date,status,start_date,name,machine,end_date,error
0,1,small,11,rf_entropy,margin,0,0,0,2024-06-13 10:35:24,created,None,None,None,None,None
1,2,small,11,svm_rbf,margin,0,0,0,2024-06-13 10:35:24,created,None,None,None,None,None
2,3,small,11,rf_entropy,random,0,0,0,2024-06-13 10:35:24,created,None,None,None,None,None
3,4,small,11,svm_rbf,random,0,0,0,2024-06-13 10:35:24,created,None,None,None,None,None




Contents of table sqlite_sequence:


,name,seq
0,results,4




Contents of table results__accuracy_log:


,ID,experiment_id,timestamp,model_dict




Contents of table results__labeling_log:


,ID,experiment_id,timestamp,data_dict




Contents of table results_codecarbon:


,ID,experiment_id,codecarbon_timestamp,project_name,run_id,duration_seconds,emissions_kg,emissions_rate_kg_sec,cpu_power_watt,gpu_power_watt,...,cpu_model,gpu_count,gpu_model,longitude,latitude,ram_total_size,tracking_mode,on_cloud,power_usage_efficiency,offline_mode


In [7]:
run(run_setup=False, reset_experiments=False)

2024-06-13 10:35:24,700  | py-experimenter - WARNING  | No values given for keyfield setting_name
2024-06-13 10:35:24,702  | py-experimenter - WARNING  | No values given for keyfield openml_id
2024-06-13 10:35:24,705  | py-experimenter - WARNING  | No values given for keyfield learner_name
2024-06-13 10:35:24,708  | py-experimenter - WARNING  | No values given for keyfield query_strategy_name
2024-06-13 10:35:24,710  | py-experimenter - WARNING  | No values given for keyfield test_split_seed
2024-06-13 10:35:24,711  | py-experimenter - WARNING  | No values given for keyfield train_split_seed
2024-06-13 10:35:24,712  | py-experimenter - WARNING  | No values given for keyfield seed
2024-06-13 10:35:24,712  | py-experimenter - INFO     | Found 7 keyfields
2024-06-13 10:35:24,714  | py-experimenter - WARNING  | No resultfields given
2024-06-13 10:35:24,715  | py-experimenter - INFO     | Found 2 logtables
2024-06-13 10:35:24,716  | py-experimenter - INFO     | Found logtable results__accur

/home/valentin/Documents/Projects/2024/24_03_MetaAL/ActiveLearningPipelines/venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/valentin/Documents/Projects/2024/24_03_MetaAL/ActiveLearningPipelines/venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/valentin/Documents/Projects/2024/24_03_MetaAL/ActiveLearningPipelines/venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use 

[codecarbon INFO @ 10:35:44] Energy consumed for RAM : 0.000002 kWh. RAM Power : 11.630139827728273 W
[codecarbon INFO @ 10:35:44] Energy consumed for all GPUs : 0.000019 kWh. Total GPU Power : 96.63183722214282 W
[codecarbon INFO @ 10:35:44] Energy consumed for all CPUs : 0.000008 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 10:35:44] 0.000029 kWh of electricity used since the beginning.
[codecarbon WARNING @ 10:35:45] Invalid gpu_ids format. Expected a string or a list of ints.
[codecarbon INFO @ 10:35:45] [setup] RAM Tracking...
[codecarbon INFO @ 10:35:45] [setup] GPU Tracking...
[codecarbon INFO @ 10:35:45] Tracking Nvidia GPU via pynvml
[codecarbon INFO @ 10:35:45] [setup] CPU Tracking...
[codecarbon WARNING @ 10:35:45] No CPU tracking mode found. Falling back on CPU constant mode.
[codecarbon WARNING @ 10:35:46] We saw that you have a 12th Gen Intel(R) Core(TM) i7-12700H but we don't know it. Please contact us.
[codecarbon INFO @ 10:35:46] CPU Model on constant consumption m

### display (filled) tables

In [8]:
# Specify the path to your .db file
db_path = "ALPBenchmark.db"

# Connect to the database
conn = sqlite3.connect(db_path)

# Create a cursor object to interact with the database
cursor = conn.cursor()

# Get the list of tables in the database
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()

# Display the contents of each table
for table_name in tables:
    table_name = table_name[0]
    print(f"Contents of table {table_name}:")
    query = f"SELECT * FROM {table_name}"
    df = pd.read_sql_query(query, conn)
    display(df)
    print("\n")

# Close the connection
conn.close()

Contents of table results:


,ID,setting_name,openml_id,learner_name,query_strategy_name,test_split_seed,train_split_seed,seed,creation_date,status,start_date,name,machine,end_date,error
0,1,small,11,rf_entropy,margin,0,0,0,2024-06-13 10:35:24,done,2024-06-13 10:35:24,PyExperimenter,valentin-XPS-15-9520,2024-06-13 10:35:34,None
1,2,small,11,svm_rbf,margin,0,0,0,2024-06-13 10:35:24,done,2024-06-13 10:35:34,PyExperimenter,valentin-XPS-15-9520,2024-06-13 10:35:39,None
2,3,small,11,rf_entropy,random,0,0,0,2024-06-13 10:35:24,done,2024-06-13 10:35:44,PyExperimenter,valentin-XPS-15-9520,2024-06-13 10:35:54,None
3,4,small,11,svm_rbf,random,0,0,0,2024-06-13 10:35:24,done,2024-06-13 10:35:39,PyExperimenter,valentin-XPS-15-9520,2024-06-13 10:35:44,None




Contents of table sqlite_sequence:


,name,seq
0,results,4
1,results__labeling_log,4
2,results__accuracy_log,4
3,results_codecarbon,4




Contents of table results__accuracy_log:


,ID,experiment_id,timestamp,model_dict
0,1,1,2024-06-13 10:35:34,"{""0"": {""iteration"": 0, ""test_f1"": 0.73535601, ..."
1,2,2,2024-06-13 10:35:39,"{""0"": {""iteration"": 0, ""test_f1"": 0.78834966, ..."
2,3,4,2024-06-13 10:35:44,"{""0"": {""iteration"": 0, ""test_f1"": 0.78834966, ..."
3,4,3,2024-06-13 10:35:54,"{""0"": {""iteration"": 0, ""test_f1"": 0.72303261, ..."




Contents of table results__labeling_log:


,ID,experiment_id,timestamp,data_dict
0,1,1,2024-06-13 10:35:34,"{""0"": {""iteration"": 0, ""len_X_sel"": 30, ""len_X..."
1,2,2,2024-06-13 10:35:39,"{""0"": {""iteration"": 0, ""len_X_sel"": 30, ""len_X..."
2,3,4,2024-06-13 10:35:44,"{""0"": {""iteration"": 0, ""len_X_sel"": 30, ""len_X..."
3,4,3,2024-06-13 10:35:54,"{""0"": {""iteration"": 0, ""len_X_sel"": 30, ""len_X..."




Contents of table results_codecarbon:


,ID,experiment_id,codecarbon_timestamp,project_name,run_id,duration_seconds,emissions_kg,emissions_rate_kg_sec,cpu_power_watt,gpu_power_watt,...,cpu_model,gpu_count,gpu_model,longitude,latitude,ram_total_size,tracking_mode,on_cloud,power_usage_efficiency,offline_mode
0,1,1,2024-06-13T10:35:34,codecarbon,6c966ab3-907e-4453-8f34-b949de6f6156,5.433720,0.000035,0.000006,42.5,5.505503,...,12th Gen Intel(R) Core(TM) i7-12700H,1.0,1 x NVIDIA GeForce RTX 3050 Ti Laptop GPU,11.5683,48.1663,31.013706,machine,N,1.0,0
1,2,2,2024-06-13T10:35:39,codecarbon,5036874c-b7e4-49bc-9035-173ba3174564,0.633812,0.000006,0.000010,42.5,39.861727,...,12th Gen Intel(R) Core(TM) i7-12700H,1.0,1 x NVIDIA GeForce RTX 3050 Ti Laptop GPU,11.5683,48.1663,31.013706,machine,N,1.0,0
2,3,4,2024-06-13T10:35:44,codecarbon,5bd67857-aed2-45b3-8e95-c49d7815bf7b,0.718391,0.000011,0.000016,42.5,96.631837,...,12th Gen Intel(R) Core(TM) i7-12700H,1.0,1 x NVIDIA GeForce RTX 3050 Ti Laptop GPU,11.5683,48.1663,31.013706,machine,N,1.0,0
3,4,3,2024-06-13T10:35:54,codecarbon,c85753de-54ea-4380-bb18-e066f35befa4,5.094018,0.000032,0.000006,42.5,5.289665,...,12th Gen Intel(R) Core(TM) i7-12700H,1.0,1 x NVIDIA GeForce RTX 3050 Ti Laptop GPU,11.5683,48.1663,31.013706,machine,N,1.0,0
